In [1]:
import json
import sys
import re

# function reading and writing the json file 

In [2]:
def read_json_from_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)
   
def save_json_to_file(data, filename):
    with open(filename, "w") as f:
        json.dump(data, f, indent=2)

# removing the Keys 

In [3]:
def filter_json_remove_slash_key_value(data, pattern="/key"):
    """
    Recursively removes keys from a dictionary if they contain the given pattern.

    """
    if not isinstance(data, dict):
        return data  # If data is not a dictionary, return as is
    
    new_dict = {}
    for k, v in data.items():
        if pattern not in k:  # Exclude keys that contain the pattern
            if isinstance(v, dict):
                new_dict[k] = filter_json_remove_slash_key_value(v, pattern)  # Recurse for nested dict
            else:
                new_dict[k] = v  # Keep the value as is
    
    return new_dict


In [4]:
def filter_json_remove_system_keys(data):
    # Ensure we only modify the 'System' key if it exists
    if 'System' in data:
        # List of keys to remove from 'System'
        keys_to_remove = ['Buffers', 'Transition', 'Flush', 'Tmp']
        
        # Loop through the list and remove each key from 'System' if it exists
        for key in keys_to_remove:
            data['System'].pop(key, None)
    # Filter out entries that have "last_written" in the key
    if isinstance(data, list):  # Check if data is a list of dictionaries
        data = [entry for entry in data if "/last_written" not in entry.get("key", "")]

    return data

In [5]:
def filter_json_remove_legacytrigger(data):
    """
    This function recursively removes keys that match the pattern "/Detectors/DetXX/Settings/LegacyTrigger".
    """
    def recursive_trim(d):
        if isinstance(d, dict):
            # Identify keys matching the specific pattern
            keys_to_remove = [key for key in d if "/Detectors/Det" in key and "/Settings/LegacyTrigger" in key]
            # Remove the matching keys
            for key in keys_to_remove:
                del d[key]
            # Recursively process nested dictionaries or lists
            for key, value in d.items():
                if isinstance(value, (dict, list)):  # If the value is a dictionary or list, apply recursion
                    recursive_trim(value)

    # Call the recursive trim function on the data
    recursive_trim(data)
    
    return data


In [6]:
def find_differences(start_obj, end_obj):
    def recursive_diff(start, end):
        diff = {}
        for key in end.keys():
            if key not in start:
                # Key is new in end_obj
                diff[key] = end[key]
            elif isinstance(start[key], dict) and isinstance(end[key], dict):
                # Recursively check nested dictionaries
                nested_diff = recursive_diff(start[key], end[key])
                if nested_diff:  # Only add if there are differences
                    diff[key] = nested_diff
            elif isinstance(start[key], list) and isinstance(end[key], list):
                # Compare lists directly
                if start[key] != end[key]:
                    diff[key] = end[key]
            elif start[key] != end[key]:
                # Value has changed
                diff[key] = end[key]
        return diff

    return recursive_diff(start_obj, end_obj)


# Saving into Json

In [8]:
json2 = read_json_from_file('24250227_135126_end_original.json')
json2 = filter_json_remove_slash_key_value(json2)
save_json_to_file(json2, "24250227_135126_end_remove_slash_key_value.json")
json2 = filter_json_remove_system_keys(json2)
save_json_to_file(json2, "24250227_135126_end_remove_system_keys_except_clients.json")

json2 = filter_json_remove_legacytrigger(json2)
save_json_to_file(json2, "24250227_135126_end_remove_legacytrigger.json")

print("saving end Json" )

saving end Json


In [9]:
json2 = read_json_from_file('24250227_135126_start_original.json')
json2 = filter_json_remove_slash_key_value(json2)
save_json_to_file(json2, "24250227_135126_start_remove_slash_key_value.json")
json2 = filter_json_remove_system_keys(json2)
save_json_to_file(json2, "24250227_135126_start_remove_system_keys_except_clients.json")

json2 = filter_json_remove_legacytrigger(json2)
save_json_to_file(json2, "24250227_135126_start_remove_legacytrigger.json")
print("saving start Json" )

saving start Json


# Finding the difference in end json with the start after allabove filerting applied

In [10]:
json2 = read_json_from_file('24250227_135126_end_original.json')
json2 = filter_json_remove_slash_key_value(json2)
json2 = filter_json_remove_system_keys(json2)
json2 = filter_json_remove_legacytrigger(json2)
# json2 = filter_json_differ_Readback_Setting(json2)
json1 = read_json_from_file('24250227_135126_start_original.json')
json1 = filter_json_remove_slash_key_value(json1)
json1 = filter_json_remove_system_keys(json1)
finddifference=find_differences(json1,json2)
finddifference

{'Logger': {'Channels': {'0': {'Settings': {'Current filename': '.RUN00075_DUMP0001.mid.gz'},
    'Statistics': {'Disk level': 0.001500701734860943,
     'Bytes written': 4489610,
     'Files written': 20818,
     'Events written': 361,
     'Bytes written total': 17319883542959.0,
     'Bytes written subrun': 2244248,
     'Bytes written uncompressed': 69735101}}}},
 'System': {'Clients': {'2049309': {'Run state': 1},
   '2049311': {'Run state': 1}}},
 'Runinfo': {'Stop time': 'Thu Feb 27 13:53:03 2025',
  'Stop time binary': '0x67c0b48f',
  'Transition in progress': 0},
 'Equipment': {'EBlvl': {'Statistics': {'Events sent': 56}},
  'EBuilder': {'Statistics': {'Events sent': 355},
   'Special dump numbers': {'Last BORR': 1, 'Last BORTS': 1}},
  'L2Trigger': {'Statistics': {'Events sent': 86}},
  'readoutfe_SDU': {'Statistics': {'Events sent': 0}},
  'readoutveto03': {'Statistics': {'Events sent': 10}},
  'L2TriggerHistory': {'Statistics': {'Events sent': 10}},
  'readouthistory03': {'

In [12]:
print("saving different in Json format")
save_json_to_file(finddifference, "24250227_135126_diff_end_with_start_ODB.json")

saving different in Json format


In [43]:
import os

# Define file names corresponding to each step with sizes in KB
file_sizes_on_disk = {
    "Original_json": 11785291/1024,
    "Json_removing_slash_key_values": 7795067 / 1024,
    "Json_removing_system_keys_except_clients": 7475285 / 1024,
    "Json_removing_legacytrigger": 7475285 / 1024,
    "Json_difference_in_end_ODB_with_start": 7955/1024
}
print()
# Print table header with better formatting
print(f"{'Action (odb end json for series=24250227_135126)':<50}{'Disk Size (KB)':>20}")
print("=" * 75)

# Print each file's disk size in a neatly formatted way
for file_name, size_kb in file_sizes_on_disk.items():
    print(f"{file_name:<50}{size_kb:>20.2f}")
print()
print()
print()


Action (odb end json for series=24250227_135126)        Disk Size (KB)
Original_json                                                 11509.07
Json_removing_slash_key_values                                 7612.37
Json_removing_system_keys_except_clients                       7300.08
Json_removing_legacytrigger                                    7300.08
Json_difference_in_end_ODB_with_start                             7.77





In [2]:
# Define file names corresponding to each step with sizes in KB
file_sizes_on_disk = {
    "Original_json": 11782529/1024,
    "Json_removing_slash_key_values": 7795110 / 1024,
    "Json_removing_system_keys_except_clients": 7475284 / 1024,
    "Json_removing_legacytrigger": 7475284 / 1024,
    
}
print()
# Print table header with better formatting
print(f"{'Action (odb start json for series=24250227_135126)':<50}{'Disk Size (KB)':>20}")
print("=" * 75)

# Print each file's disk size in a neatly formatted way
for file_name, size_kb in file_sizes_on_disk.items():
    print(f"{file_name:<50}{size_kb:>20.2f}")
print()
print()
print()


Action (odb start json for series=24250227_135126)      Disk Size (KB)
Original_json                                                 11506.38
Json_removing_slash_key_values                                 7612.41
Json_removing_system_keys_except_clients                       7300.08
Json_removing_legacytrigger                                    7300.08



